In [70]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()


Successfully saved authorization token.


In [71]:
import ee

In [72]:
import os
import pandas as pd
import sys
import glob
import numpy as np
import matplotlib.pyplot as plt

In [73]:
from utils_pygee import *

In [74]:
from importlib import reload
reload(sys.modules['utils_pygee'])
from utils_pygee import *

# Parameters

In [75]:
States = ee.FeatureCollection("TIGER/2018/States")
US_States = States.filter(ee.Filter.inList('NAME', ['North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Iowa', 'Missouri', 'Illinois', 'Indiana', 'Ohio']))
#US_States = States.filter(ee.Filter.inList('NAME', ['North Dakota', 'South Dakota']))

In [76]:
state = US_States.filter(ee.Filter.eq('NAME', 'Illinois'))
# Define a small rectangle within Illinois for testing
test_region = ee.Geometry.Rectangle([-89.5, 39.5, -88.5, 40.5])  # Define the rectangle by [minLon, minLat, maxLon, maxLat]

In [77]:
# start end date
start_date = '2021-01-01'
end_date = '2023-11-01'

# Dataset

In [24]:
#https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MCD15A3H
FparLai_collection = (ee.ImageCollection('MODIS/061/MCD15A3H')
                  .filterDate(ee.Date(start_date), ee.Date(end_date)))

In [60]:
#https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MCD43A4
modis_collection = (ee.ImageCollection('MODIS/061/MCD43A4')
                     .filterDate(ee.Date(start_date), ee.Date(end_date)))

In [11]:
#https://developers.google.com/earth-engine/datasets/catalog/NOAA_CFSV2_FOR6H#description
cfsv2_collection = (ee.ImageCollection('NOAA/CFSV2/FOR6H')
                        .filterDate(ee.Date(start_date), ee.Date(end_date)))

# For loop for small peroiods for cfsv2 collection

In [79]:
# Generate sdate and edate, which is Every 6 months from 2003-01-01 to 2023-01-01
start_date = '2022-01-01'
end_date = '2023-01-01'
sdates = pd.date_range(start_date, end_date, freq='6MS')
#edate = pd.date_range(start_date, end_date, freq='6M')

#print(edate)
for sdate in sdates:
    edate = sdate + pd.DateOffset(months=6)
    print(sdate,edate)
    cfsv2_collection = (ee.ImageCollection('NOAA/CFSV2/FOR6H')
                        .filterDate(ee.Date(sdate), ee.Date(edate)))
    cfsv2_masked = cfsv2_collection.map(mask_noncrop)
    cfsv2_filtered = cfsv2_masked

    State_list = ['North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Iowa', 'Missouri', 'Illinois', 'Indiana', 'Ohio']
    for state in State_list:
        print("--------------------{}--------------------".format(state))
        aoi = States.filter(ee.Filter.eq('NAME', state))
        generate_ts_df(cfsv2_filtered, aoi, 'Maximum_temperature_height_above_ground_6_Hour_Interval', freq='1M', func= "median",Output=True, dir_out=r"A:\US_CornSoy_ClassificationYield\GEEData\State_Monitor")
        print("Tmax done")
        generate_ts_df(cfsv2_filtered, aoi, 'Minimum_temperature_height_above_ground_6_Hour_Interval', freq='1M', func = "median",Output=True, dir_out=r"A:\US_CornSoy_ClassificationYield\GEEData\State_Monitor")
        print("Tmin done")
        generate_ts_df(cfsv2_filtered, aoi, 'Volumetric_Soil_Moisture_Content_depth_below_surface_layer_5_cm', freq='1M', func = "median",Output=True, dir_out=r"A:\US_CornSoy_ClassificationYield\GEEData\State_Monitor")
        print("Soil Moisture 5cm done")
        generate_ts_df(cfsv2_filtered, aoi, 'Volumetric_Soil_Moisture_Content_depth_below_surface_layer_25_cm', freq='1M',func = "median", Output=True, dir_out=r"A:\US_CornSoy_ClassificationYield\GEEData\State_Monitor")
        print("Soil Moisture 25cm done")
        generate_ts_df(cfsv2_filtered, aoi, 'Precipitation_rate_surface_6_Hour_Average', freq='1M', func = "sum",Output=True, dir_out=r"A:\US_CornSoy_ClassificationYield\GEEData\State_Monitor")
        print("Precipitation done")


2022-01-01 00:00:00 2022-07-01 00:00:00
--------------------North Dakota--------------------
Tmax done
Tmin done
Soil Moisture 5cm done
Soil Moisture 25cm done
Precipitation done
--------------------South Dakota--------------------
Tmax done
Tmin done
Soil Moisture 5cm done
Soil Moisture 25cm done
Precipitation done
--------------------Nebraska--------------------
Tmax done
Tmin done
Soil Moisture 5cm done
Soil Moisture 25cm done
Precipitation done
--------------------Kansas--------------------
Tmax done
Tmin done
Soil Moisture 5cm done
Soil Moisture 25cm done
Precipitation done
--------------------Iowa--------------------
Tmax done
Tmin done
Soil Moisture 5cm done
Soil Moisture 25cm done
Precipitation done
--------------------Missouri--------------------
Tmax done
Tmin done
Soil Moisture 5cm done
Soil Moisture 25cm done
Precipitation done
--------------------Illinois--------------------
Tmax done
Tmin done
Soil Moisture 5cm done
Soil Moisture 25cm done
Precipitation done
-------------

# Preprocess

## Apply Non-crop mask

In [25]:
FparLai_masked = FparLai_collection.map(mask_noncrop)
FparLai_filtered = FparLai_masked.map(lambda image: QAfilter(image, 'FparLai_QC', 0, 0))

In [61]:
modis_masked = modis_collection.map(mask_noncrop)
modis_filtered = modis_masked.map(lambda image: QAfilter(image, 'BRDF_Albedo_Band_Mandatory_Quality_Band1', 0, 0))

In [14]:
cfsv2_masked = cfsv2_collection.map(mask_noncrop)
cfsv2_filtered = cfsv2_masked # There is no quality band for CFSV2

## Add VIs

In [62]:
modis_filtered = modis_filtered.map(addNDVI_MODIS)
modis_filtered = modis_filtered.map(addEVI_MODIS)

# Extract time series

## CFSV2 
Due to its 6-hour frequency, it requires longer time to run and needs to reduce start and end date interval to avoid computational overload

In [ ]:
State_list = ['North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Iowa', 'Missouri', 'Illinois', 'Indiana', 'Ohio']
for state in State_list:
    print("--------------------{}--------------------".format(state))
    aoi = States.filter(ee.Filter.eq('NAME', state))
    generate_ts_df(cfsv2_filtered, aoi, 'Maximum_temperature_height_above_ground_6_Hour_Interval', freq='1M', func= "median",Output=True, dir_out=r"A:\US_CornSoy_ClassificationYield\GEEData\State_Monitor")
    print("Tmax done")
    generate_ts_df(cfsv2_filtered, aoi, 'Minimum_temperature_height_above_ground_6_Hour_Interval', freq='1M', func = "median",Output=True, dir_out=r"A:\US_CornSoy_ClassificationYield\GEEData\State_Monitor")
    print("Tmin done")
    generate_ts_df(cfsv2_filtered, aoi, 'Volumetric_Soil_Moisture_Content_depth_below_surface_layer_5_cm', freq='1M', func = "median",Output=True, dir_out=r"A:\US_CornSoy_ClassificationYield\GEEData\State_Monitor")
    print("Soil Moisture 5cm done")
    generate_ts_df(cfsv2_filtered, aoi, 'Volumetric_Soil_Moisture_Content_depth_below_surface_layer_25_cm', freq='1M',func = "median", Output=True, dir_out=r"A:\US_CornSoy_ClassificationYield\GEEData\State_Monitor")
    print("Soil Moisture 25cm done")
    generate_ts_df(cfsv2_filtered, aoi, 'Precipitation_rate_surface_6_Hour_Average', freq='1M', func = "sum",Output=True, dir_out=r"A:\US_CornSoy_ClassificationYield\GEEData\State_Monitor")
    print("Precipitation done")
    

## MODIS VIs

In [63]:
State_list = ['North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Iowa', 'Missouri', 'Illinois', 'Indiana', 'Ohio']
for state in State_list:
    print("--------------------{}--------------------".format(state))
    aoi = States.filter(ee.Filter.eq('NAME', state))
    generate_ts_df(modis_filtered, aoi, 'NDVI', freq='1M', func= "median", Output = True, dir_out = r"A:\US_CornSoy_ClassificationYield\GEEData\State_Monitor")
    print('NDVI done')
    generate_ts_df(modis_filtered, aoi, 'EVI', freq='1M', func = "median", Output = True, dir_out = r"A:\US_CornSoy_ClassificationYield\GEEData\State_Monitor")
    print('EVI done')

--------------------North Dakota--------------------
NDVI done
EVI done
--------------------South Dakota--------------------
NDVI done
EVI done
--------------------Nebraska--------------------
NDVI done
EVI done
--------------------Kansas--------------------
NDVI done
EVI done
--------------------Iowa--------------------
NDVI done
EVI done
--------------------Missouri--------------------
NDVI done
EVI done
--------------------Illinois--------------------
NDVI done
EVI done
--------------------Indiana--------------------
NDVI done
EVI done
--------------------Ohio--------------------
NDVI done
EVI done


## LAI and FPAR

In [26]:
State_list = ['North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Iowa', 'Missouri', 'Illinois', 'Indiana', 'Ohio']
for state in State_list:
    print("--------------------{}--------------------".format(state))
    aoi = States.filter(ee.Filter.eq('NAME', state))
    generate_ts_df(FparLai_filtered, aoi, 'Lai', freq='1M', func= "median", Output = True, dir_out = r"A:\US_CornSoy_ClassificationYield\GEEData\State_Monitor")
    print('Lai done')
    generate_ts_df(FparLai_filtered, aoi, 'Fpar', freq='1M', func = "median", Output = True, dir_out = r"A:\US_CornSoy_ClassificationYield\GEEData\State_Monitor")
    print('Fpar done')

--------------------North Dakota--------------------
Lai done
Fpar done
--------------------South Dakota--------------------
Lai done
Fpar done
--------------------Nebraska--------------------
Lai done
Fpar done
--------------------Kansas--------------------
Lai done
Fpar done
--------------------Iowa--------------------
Lai done
Fpar done
--------------------Missouri--------------------
Lai done
Fpar done
--------------------Illinois--------------------
Lai done
Fpar done
--------------------Indiana--------------------
Lai done
Fpar done
--------------------Ohio--------------------
Lai done
Fpar done
